In [1]:
import urllib.request
import ssl
import json
import time
import pandas as pd
from datetime import datetime
import numpy as np
import plotly
import ffn
import finquant
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
from datetime import date

#df_ticker = pd.DataFrame.from_dict(cg.get_coins_list())
df_ticker = pd.read_csv('tickers.csv')
#df_ticker.to_csv("tickers.csv")

In [2]:
def get_coin_prices(coin_list):
    frames = []
    for coin_name in coin_list:
        df_price = pd.DataFrame.from_dict(cg.get_coin_market_chart_by_id(id=coin_name, vs_currency='usd', days=364))
        df = pd.DataFrame()
        df[coin_name] = [i[1] for i in df_price["prices"]]
        df.index = [datetime.fromtimestamp(int(i[0])/1000.0) for i in df_price["prices"]]
        frames.append(df)
    df = pd.concat(frames, axis=1)
    df.index.name = "Date"
    return df


def get_coin_name_list(symbol_list):
    l = []
    for c in symbol_list:
        c = c.lower()
        name = df_ticker[df_ticker["symbol"] == c]
    l.append(name)
    return l

coin_list = ["bitcoin", "ethereum", "cardano", "litecoin", "solana", "pancakeswap-token", "ripple", "polkadot", "binancecoin"]
symbol_list = ['ADA', 'ETH', 'AGI', 'LINK', 'SOL', 'LTC', 'COTI', 'ETC' '1inch']
print(get_coin_name_list(symbol_list))

df = get_coin_prices(coin_list).dropna()
print(df)

info = cg.get_coin_market_chart_by_id(id=coin_list[0], vs_currency='usd', days=364)
print(info)
#returns = df.to_returns()

[Empty DataFrame
Columns: [Unnamed: 0, id, symbol, name]
Index: []]
                          bitcoin     ethereum   cardano    litecoin  \
Date                                                                   
2020-10-22 21:00:00  12951.044060   413.201890  0.110356   54.304243   
2020-10-23 21:00:00  12928.072296   408.889489  0.107934   55.332143   
2020-10-24 21:00:00  13106.046019   411.959218  0.107971   59.077498   
2020-10-25 21:00:00  13021.410842   406.157094  0.106272   58.795938   
2020-10-26 21:00:00  13060.792212   393.382908  0.103267   56.859877   
...                           ...          ...       ...         ...   
2021-10-15 21:00:00  61809.023944  3885.641765  2.225648  189.335150   
2021-10-16 21:00:00  61165.635642  3854.498461  2.187229  187.151126   
2021-10-17 21:00:00  61583.774440  3854.223687  2.157823  183.599868   
2021-10-18 21:00:00  62017.972725  3752.618727  2.128426  185.533037   
2021-10-19 21:00:00  64517.648560  3884.587295  2.115809  188.962532

In [3]:
returns = ffn.core.to_returns(df)
def calc_stats_from_df(df):
    perf = ffn.core.calc_stats(df)
    stats = perf.stats.T
    stats = stats.sort_values(by="monthly_sharpe", ascending=False)
    return stats

In [4]:
def calc_weights(returns):
    weights_df = pd.DataFrame()
    weights_df["risk_parity_weights"] = ffn.core.calc_erc_weights(returns, initial_weights=None, risk_weights=None, covar_method='standard', risk_parity_method='ccd', maximum_iterations=1000, tolerance=1e-03)
    weights_df["inv_vol_weights"] = ffn.core.calc_inv_vol_weights(returns)
    weights_df["mean_var_weights"] = ffn.calc_mean_var_weights(returns, rf=0.05, covar_method='standard')
    return weights_df
weights_df = calc_weights(returns)
print(weights_df)

                   risk_parity_weights  inv_vol_weights  mean_var_weights
bitcoin                       0.173807         0.181870      1.040834e-17
ethereum                      0.123697         0.138211      2.162390e-17
cardano                       0.114127         0.111305      0.000000e+00
litecoin                      0.111742         0.123378      0.000000e+00
solana                        0.104366         0.085954      6.383782e-16
pancakeswap-token             0.084126         0.078901      1.000000e+00
ripple                        0.096306         0.084620      3.957794e-16
polkadot                      0.095605         0.097060      5.420253e-16
binancecoin                   0.096225         0.098702      9.020562e-17


In [5]:
def back_testing(df, strategy_name):
    df_rel = df.div(df.iloc[0])
    for col in df.columns:
        df_rel[col] = df_rel[col] * weights_df[strategy_name][col]
    overall = []
    for i in range(len(df_rel.index)):
        overall.append(np.sum(df_rel.iloc[i]))
    return overall

df_rel = df.div(df.iloc[0])
df_rel["risk_parity_weights"] = back_testing(df, "risk_parity_weights")
df_rel["inv_vol_weights"] = back_testing(df, "inv_vol_weights")
df_rel["mean_var_weights"] = back_testing(df, "mean_var_weights")

In [6]:
df_stats = calc_stats_from_df(df_rel)
df_stats.to_csv('backtesting.csv')

In [7]:
import finquant

finquant.efficient_frontier.EfficientFrontier(mean_returns, cov_matrix, risk_free_rate=0.005, freq=252, method='SLSQP')

AttributeError: module 'finquant' has no attribute 'efficient_frontier'

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

fig = go.Figure()
for i in df_rel.columns:
    
    if i in ["risk_parity_weights", "inv_vol_weights"]:
        fig.add_trace(go.Scatter(x=df_rel.index, y=df_rel[i],
                    mode='lines',
                    name=i))
    else:
        fig.add_trace(go.Scatter(x=df_rel.index, y=df_rel[i],
                    mode='lines',
                    name=i, opacity=0.2))
        #plt.plot(df_rel.index, df_rel[i], alpha=0.2, linewidth=0.5)
#plt.legend(loc='best', bbox_to_anchor=(0.5, 0., 0.5, 0.5))
#fig.update_layout("May 2020 - May 2021 Investiment strategies backtesting")
fig.show()
fig.write_html("backtesting.html")

In [ ]:
cov_mat = np.cov(df.to_numpy())
farm_yields = pd.read_csv('farm_yields.csv')
print(farm_yields)

     TOKEN1 TOKEN2       DPR     INFL1     INFL2         INFL            IL
0      LAVA    NaN  0.042267  0.000000       NaN      #VALUE!       #VALUE!
1     OLIVE   USDT  0.028569  0.000000  0.000164  1.000163528  -3.34215E-09
2      AVAX  OLIVE  0.023602  0.000000  0.000000            1             0
3      BUSD   MASH  0.021211  0.000164  0.000000  0.999836498  -3.34215E-09
4      BUSD    MDS  0.020059  0.000164  0.000000  0.999836498  -3.34215E-09
..      ...    ...       ...       ...       ...          ...           ...
285  BANANA    NaN -0.008682  0.014300       NaN      #VALUE!       #VALUE!
286   SWAMP    NaN -0.011697  0.022831       NaN      #VALUE!       #VALUE!
287    BUSD  SPACE -0.016286  0.000164  0.062321  1.062147497  -0.000454231
288     BNB  SPACE -0.020641 -0.000072  0.062321  1.062397266  -0.000457779
289   XBLZD    NaN -0.022503  0.032817       NaN      #VALUE!       #VALUE!

[290 rows x 7 columns]


In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
# load dataset

X = df["bitcoin"]
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# walk-forward validation
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	#print('predicted=%f, expected=%f' % (yhat, obs))
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



Test RMSE: 1602.851


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\ipykernel_launcher.py:31: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=test.index, y=test.values,
    name="observed"
))

fig.add_trace(go.Scatter(
    x=test.index, y=predictions,
    name="predicted"
))
fig.update_layout(title= "BTC price prediction")
fig.show()


NameError: name 'test' is not defined